## [한국수자원공사] 가뭄 기상관측 openAPI

##### 목적 : 가뭄 관련 기상 관측소별 3개년 데이터 수집

In [1]:
### 사용함수 선언
import pandas as pd
import requests
import datetime as dt

from bs4 import BeautifulSoup
from urllib.parse import urlencode, quote_plus, unquote

### 1. XML API 요청을 위한 URL 생성 (openAPI 참조문서의 요청명세를 확인하여 필수 queryString 구성 및 인코딩)
## 가뭄 기상관측 openAPI 조회를 위한 queryString 필수 값으로 관측 시설물 코드 리스트 생성
facilityCodeDf = pd.read_csv("../dataset/facilityCode_1016.csv")
facilityCodeList = facilityCodeDf.cd.tolist()
facilityNameList = facilityCodeDf.cdnm.tolist()

baseUrl = "http://apis.data.go.kr/B500001/drghtKmaOper/operInfoList"
currentYear = dt.datetime.now().year
yearList = [str(currentYear-3), str(currentYear-2), str(currentYear-1)] # 이전 3개년 검색에 필요한 데이터
serviceKey = "2AlYmSdbDUuKGC0DivNISdLDUqJewAoneYBBTMwCjqcQz8cIiQXSp0je68IfpEVPg6+J2f8OeNy3lak986T1rQ==" 
numOfRows = "9999" 
pageNo = "1"

for i in range (0, len(facilityCodeList)): # 각의 관측소를 기준으로 데이터프레임을 생성하고자 함
    obsrvtCd = facilityCodeList[i]
    region = facilityNameList[i] # csv저장시 파일명에 사용할 데이터
    for j in range(0, len(yearList)):
        year = yearList[j]
        queryParam2 = '?' + urlencode(
            {
                quote_plus('serviceKey') : serviceKey, # 인증키
                quote_plus('pageNo'): pageNo, 
                quote_plus('numOfRows') : numOfRows, # 한 페이지에 노출되는 결과 값 (관측소에서 일일 데이터 측정, 따라서 연간 최대 365개)                quote_plus('pageNo') : pageNo, ## 페이지는 1페이지로 고정
                quote_plus('obsrvtCd') : obsrvtCd, # 관측소코드 값
                quote_plus('stDt') : year + "0101", 
                quote_plus('edDt') : year + "1231"
            }
        )
        obsTargetUrl = baseUrl + queryParam2 

        ### 2. XML API 요청 및 응답받기(BeautifulSoup으로 편집)
        resp = requests.get(obsTargetUrl)
        resp.encoding = "utf-8"
        xml = resp.text
        bsXml = BeautifulSoup(xml, "lxml-xml")
        
        ### 3. DataFrame으로 만들 필요한 데이터 수집
        ## 3-1.
        rows = bsXml.findAll(name = "item")

        ## 사용할 List 선언
        rowList = [] # 전체 행을 담을 리스트(DataFrame 생성시 사용)
        columnList = [] # 각 행의 컬럼값을 담을 리스트
        nameList = ["avgpv", "avgtmp", "avgts", "avgwv", "obsrdate", "obsrvtcd"] 

        ### 3-2. 행별 컬럼값을 추출
        for k in range(0, len(rows)):
            ## 주의 : item태그의 자식태그 개수가 생성할 DataFrame의 컬럼개수와 다른 경우 발생. 따라서 DataFrame 생성시, 컬럼값과 컬럼명이 매치되지 않는 상황 발생
            columns = rows[k].findAll() 
            if (len(columns) != len(nameList)): # item태그의 자식태그 개수가 생성할 DataFrame의 컬럼 개수와 다르면 columns 재정의
                newColumns = [] # 생성할 DataFrame의 컬럼명을 기준으로 columns를 재정의할 리스트
                
                # 생성할 dataframe의 컬럼명을 기준으로 자식태그와 비교하여 같은 값 찾기 
                for n in range (0, len(nameList)):
                    for m in range (0, len(columns)):
                        if(nameList[n] == columns[m].name): 
                            newColumns.append(columns[m])
                            break
                        else:
                            continue
                            
                    if (len(newColumns) == n): # DataFrame 컬럼명에 맞는 태그가 없는 경우에 None처리해서 값 만들어 넣기
                        newColumns.append(None)
                    else:
                        continue
                columns = newColumns 
                
            else: # columns의 개수가 생성할 dataframe의 컬럼 개수와 같으면, 재정의 하지않고 그대로 columns 사용
                pass
            
            for l in range(0, len(columns)):
                if (columns[l] == None):
                    columnList.append(None)
                else:
                    columnList.append(columns[l].text) 
                    
            rowList.append(columnList)
            columnList = []

        ### 4. 행값과 컬럼명을 사용하여 DataFrame 생성 및 csv저장
        obsDroughtInfoDf = pd.DataFrame(rowList, columns = nameList)
        obsDroughtInfoDf.to_csv(("../dataset/observation/obsDroughtInfoDf_{}_{}_{}.csv".format(obsrvtCd, region, year)), index = False, encoding = "ms949")